In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [14]:
def obter_irradiacao_pvgis(latitude, longitude, ano=2020):
    url = "https://re.jrc.ec.europa.eu/api/seriescalc"

    parametros = {
        'lat': latitude,
        'lon': longitude,
        'startyear': ano,
        'endyear': ano,
        'outputformat': 'json',
        'angle': 0,
        'aspect': 0,
        'global': 1,
        'localtime': 1,
        'usehorizon': 1,
        'pvcalculation': 0,
        'radiation_db': 'PVGIS-SARAH2',
        'hourlyvalues': 1
    }

    resposta = requests.get(url, params=parametros)

    if resposta.status_code != 200:
        raise Exception(f"Erro na requisição: {resposta.status_code} - {resposta.text}")

    dados = resposta.json()

    irradiacao = []
    for ponto in dados['outputs']['hourly']:
        irradiacao.append({
            'Latitude': latitude,
            'Longitude': longitude,
            'Data': ponto['time'],
            'GHI': ponto['G(i)']
        })

    return pd.DataFrame(irradiacao)


# Exemplo df_ptd com possíveis repetições
df_ptd = pd.DataFrame({
    'Latitude': [-23.55, -22.90, -23.55],
    'Longitude': [-46.63, -43.20, -46.63]
})

In [15]:
df_ptd = pd.read_csv(r"C:\Users\toto9\Downloads\CP_distribPerc (1).csv")
df_ptd.head()

,fid,Código Postal 4 Dígitos,Código Postal 3 Dígitos,Código Postal Junto,Latitude,Longitude,geometry,X,Y,fid_2,Name,area_intersect,area_perc,id_codpostal,id_PTD,voronoi_codpostal_area_total
0,1,3060,610,3060610,40.351731,-8.752189,POINT (-8.752189 40.351731),-52593.610045,76073.245645,1231,20%-39%,2407,0.073217,109588,1231,3.287479e+06
1,2,3060,610,3060610,40.351731,-8.752189,POINT (-8.752189 40.351731),-52593.610045,76073.245645,1515,20%-39%,2413295,73.408704,109588,1515,3.287479e+06
2,3,3060,610,3060610,40.351731,-8.752189,POINT (-8.752189 40.351731),-52593.610045,76073.245645,1809,40%-59%,871776,26.518079,109588,1809,3.287479e+06
3,4,3060,672,3060672,40.317880,-8.762777,POINT (-8.762777 40.31788),-53519.851547,72320.703943,931,20%-39%,121238,28.977616,161264,931,4.183848e+05
4,5,3060,672,3060672,40.317880,-8.762777,POINT (-8.762777 40.31788),-53519.851547,72320.703943,1123,+100%,63622,15.206568,161264,1123,4.183848e+05


In [16]:
df_ptd_unico = df_ptd.drop_duplicates(subset=['Latitude', 'Longitude']).reset_index(drop=True)
df_ptd_unico.head()

,fid,Código Postal 4 Dígitos,Código Postal 3 Dígitos,Código Postal Junto,Latitude,Longitude,geometry,X,Y,fid_2,Name,area_intersect,area_perc,id_codpostal,id_PTD,voronoi_codpostal_area_total
0,1,3060,610,3060610,40.351731,-8.752189,POINT (-8.752189 40.351731),-52593.610045,76073.245645,1231,20%-39%,2407,0.073217,109588,1231,3.287479e+06
1,4,3060,672,3060672,40.317880,-8.762777,POINT (-8.762777 40.31788),-53519.851547,72320.703943,931,20%-39%,121238,28.977616,161264,931,4.183848e+05
2,8,3060,704,3060704,40.311975,-8.754792,POINT (-8.754792 40.311975),-52845.750572,71660.206639,131,40%-59%,18487,8.085212,167681,131,2.286523e+05
3,11,3020,523,3020523,40.277812,-8.612605,POINT (-8.612605 40.277812),-40779.730439,67791.571060,235,40%-59%,24394,0.191107,69708,235,1.276454e+07
4,26,3045,119,3045119,40.308649,-8.712562,POINT (-8.712562 40.308649),-49258.416794,71266.538560,27,40%-59%,4769907,13.284030,114296,27,3.590713e+07


In [17]:
import time

# Lista com resultados
dfs_resultados = []

for index, row in df_ptd_unico.iterrows():
    print(f"Consultando Lat: {row['Latitude']}, Lon: {row['Longitude']}")
    df_resultado = obter_irradiacao_pvgis(row['Latitude'], row['Longitude'])
    dfs_resultados.append(df_resultado)
    time.sleep(1/30)  # respeita API

# Concatenar tudo
df_irradiacao_total = pd.concat(dfs_resultados, ignore_index=True)

print(df_irradiacao_total.head())

Consultando Lat: 40.351731, Lon: -8.752189
Consultando Lat: 40.31788, Lon: -8.762777
Consultando Lat: 40.311975, Lon: -8.754792
Consultando Lat: 40.277812, Lon: -8.612605
Consultando Lat: 40.308649, Lon: -8.712562
Consultando Lat: 40.306577, Lon: -8.760361
Consultando Lat: 40.307966, Lon: -8.756799
Consultando Lat: 40.314016, Lon: -8.754496
Consultando Lat: 40.312598, Lon: -8.751206
Consultando Lat: 40.323268, Lon: -8.752914
Consultando Lat: 40.315765, Lon: -8.754506
Consultando Lat: 40.316025, Lon: -8.75262
Consultando Lat: 40.330136, Lon: -8.753295
Consultando Lat: 40.319164, Lon: -8.745897
Consultando Lat: 40.314549, Lon: -8.749601
Consultando Lat: 40.346153, Lon: -8.751142
Consultando Lat: 40.370316, Lon: -8.736746
Consultando Lat: 40.310276, Lon: -8.750003
Consultando Lat: 40.326753, Lon: -8.745911
Consultando Lat: 40.319247, Lon: -8.728558
Consultando Lat: 40.34552, Lon: -8.747258
Consultando Lat: 40.368505, Lon: -8.736363
Consultando Lat: 40.33124, Lon: -8.73702
Consultando Lat:

In [18]:
df_irradiacao_total.to_csv(r"C:\Users\toto9\Desktop\Seminário\series_temporais_irradiacao_coord_2020.csv")

In [19]:
df_irradiacao_total.head(20)

,Latitude,Longitude,Data,GHI
0,40.351731,-8.752189,20200101:0010,0.00
1,40.351731,-8.752189,20200101:0110,0.00
2,40.351731,-8.752189,20200101:0210,0.00
3,40.351731,-8.752189,20200101:0310,0.00
4,40.351731,-8.752189,20200101:0410,0.00
5,40.351731,-8.752189,20200101:0510,0.00
6,40.351731,-8.752189,20200101:0610,0.00
7,40.351731,-8.752189,20200101:0710,0.00
8,40.351731,-8.752189,20200101:0810,0.00
9,40.351731,-8.752189,20200101:0910,145.01
